In [4]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import matplotlib.animation as animation
import os

from collections import OrderedDict

## Lid Driven Cavity Problem

### Problem formulation
Navier-Stokes equations

$$\textbf{u}_t + (\nabla\cdot\textbf{u})\textbf{u} = \nu\nabla^2\textbf{u} - \nabla p$$
$$\nabla\cdot u = 0$$

### Boundary conditions
Let $\textbf{u} = (u_x, u_y)$ be the velocity field. Then at the boundaries $u_x = 0, u_y = 0$ for all walls except for the top wall, where $u_x = 1, u_y = 0$.

## Initialize the Neural Network

In [12]:
# define parameters
hidden_layers = 1
nodes_per_layer = 512

# define output directory
sim_type = 'lid_cavity'
out_dir = os.path.join('results', 'ntk','%dnode%dlayer' % (nodes_per_layer, hidden_layers), sim_type)

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

In [13]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.layers = [3] + [nodes_per_layer for _ in range(hidden_layers)] + [2]
        self.depth = len(self.layers) - 1

        self.activation = nn.Tanh

        layer_list = list()
        for i in range(self.depth - 1):
            layer_list.append(('layer_%d' % i, torch.nn.Linear(self.layers[i], self.layers[i+1])))
            layer_list.append(('activation_%d' % i, self.activation()))

        layer_list.append(('layer_%d' % (self.depth - 1), nn.Linear(self.layers[-2], self.layers[-1])))

        layerDict = OrderedDict(layer_list)
        self.model = nn.Sequential(layerDict)
        self.model.apply(self.init_weights)

    # Xavier initilization
    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0.0)

    def forward(self, x):
        out = self.model(x)
        return out

In [14]:
# Jacobian function
def jacobian(output, inputs, use_pfor=True, parallel_iterations=None):
    """Computes jacobian of `output` w.r.t. `inputs`.
    Args:
        output: A tensor.
        inputs: A tensor or a nested structure of tensor objects.
        use_pfor: If true, uses pfor for computing the jacobian. Else uses
          torch.autograd.grad.
        parallel_iterations: A knob to control how many iterations and dispatched in
          parallel. This knob can be used to control the total memory usage.
    Returns:
        A tensor or a nested structure of tensors with the same structure as
        `inputs`. Each entry is the jacobian of `output` w.r.t. to the corresponding
        value in `inputs`. If output has shape [y_1, ..., y_n] and inputs_i has
        shape [x_1, ..., x_m], the corresponding jacobian has shape
        [y_1, ..., y_n, x_1, ..., x_m]. Note that in cases where the gradient is
        sparse (IndexedSlices), jacobian function currently makes it dense and
        returns a Tensor instead. This may change in the future.
    """
    if not use_pfor:
        # Directly compute gradients using torch.autograd.grad
        grads = torch.autograd.grad(output, inputs, create_graph=True, allow_unused=True)
        return grads

    # Otherwise, use loop-based approach similar to control_flow_ops.pfor
    flat_inputs = torch.flatten(inputs)
    output_size = output.numel()
    pfor_outputs = []

    for i in range(output_size):
        y = output.reshape(-1)[i]
        grad_y = torch.autograd.grad(y, flat_inputs, create_graph=True, allow_unused=True)
        pfor_outputs.append(grad_y)

    # Reshape the output tensors to match the original shape
    for i, out in enumerate(pfor_outputs):
        if isinstance(out, torch.Tensor):
            new_shape = (output.shape + out.shape[1:])
            out = out.reshape(new_shape)
            pfor_outputs[i] = out

    return torch.stack(pfor_outputs)

## Initialize PINNS

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nu = 0.01/np.pi

class NS:
    def __init__(self, X, Y, T, u, v, kernel_size):
        self.x = torch.tensor(X, dtype=torch.float32, requires_grad=True).to(device)
        self.y = torch.tensor(Y, dtype=torch.float32, requires_grad=True).to(device)
        self.t = torch.tensor(T, dtype=torch.float32, requires_grad=True).to(device)

        self.u = torch.tensor(u, dtype=torch.float32).to(device)
        self.v = torch.tensor(v, dtype=torch.float32).to(device)

        self.kernel_size = kernel_size

        self.null = torch.zeros((self.x.shape[0], 1)).to(device)
        
        # define the neural net
        self.net = DNN().to(device)

        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=1e-3)
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=self.optimizer, gamma=0.9)
        self.mse = nn.MSELoss().to(device)

        # set kernel size for NTK computation
        self.D1, self.D2, self.D3 = self.kernel_size, self.kernel_size, self.kernel_size

        # weights on loss function: lam_u ==> IC/BC weight, lam_r ==> residual weight
        self.lam_u, self.lam_r = 1, 1

        # track loss history
        self.loss_history = []
        self.loss_history_u = []
        self.loss_history_v = []
        self.loss_history_f = []
        self.loss_history_g = []

        # track weights history
        self.lam_u_log = []
        self.lam_r_log = []
        
        # track NTK history
        self.K_u_log = []
        self.K_r_log = []

        # loss and number of iterations
        self.ls = 0
        self.iter = 0

    def get_weights_and_biases(self, model):
        weights = []
        biases = []
        for name, param in model.named_parameters():
            if 'weight' in name:
                weights.append(param.data)
            elif 'bias' in name:
                biases.append(param.data)

        return weights, biases

    def function(self, x, y, t):
        # calculates the residuals
        res = self.net(torch.hstack((x, y, t)))
        psi, p = res[:, 0:1], res[:, 1:2]

        u = torch.autograd.grad(psi, y, grad_outputs=torch.ones_like(psi), 
                                create_graph=True)[0]
        v = -1.*torch.autograd.grad(psi, x, grad_outputs=torch.ones_like(psi), 
                                    create_graph=True)[0]

        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), 
                                  create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), 
                                   create_graph=True)[0]
        u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), 
                                  create_graph=True)[0]
        u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), 
                                   create_graph=True)[0]
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), 
                                  create_graph=True)[0]

        v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), 
                                  create_graph=True)[0]
        v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), 
                                   create_graph=True)[0]
        v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), 
                                  create_graph=True)[0]
        v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), 
                                   create_graph=True)[0]
        v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), 
                                  create_graph=True)[0]

        p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(p), 
                                  create_graph=True)[0]
        p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(p), 
                                  create_graph=True)[0]

        f = u_t + u * u_x + v * u_y + p_x - nu * (u_xx + u_yy)
        g = v_t + u * v_x + v * v_y + p_y - nu * (v_xx + v_yy)

        return u, v, p, f, g
    
    def closure(self):
        self.optimizer.zero_grad()

        u_pred, v_pred, p_pred, f_pred, g_pred = self.function(self.x, self.y, self.t)

        # boundary and initial condition loss
        u_loss = self.mse(u_pred, self.u)
        v_loss = self.mse(v_pred, self.v)
        
        # residual loss
        f_loss = self.mse(f_pred, self.null)
        g_loss = self.mse(g_pred, self.null)

        # compute Jacobians
        #TODO: check correctness
        self.J_u = self.compute_jacobian(torch.hstack((u_pred, v_pred))) 
        self.J_r = self.compute_jacobian(torch.hstack((f_pred, g_pred)))

        # compute K(n)
        self.K_u = self.compute_ntk(self.J_u, self.D1, self.J_u, self.D2)
        self.K_r = self.compute_ntk(self.J_r, self.D1, self.J_r, self.D2)

        # compute lam_u, lam_r
        trace_K = np.trace(self.K_u) + np.trace(self.K_r)
        self.lam_u = trace_K / np.trace(self.K_u)
        self.lam_r = trace_K / np.trace(self.K_r)

        # compute total loss
        self.ls = self.lam_u * (u_loss + v_loss) + self.lam_r * (f_loss + g_loss)

        # track loss history
        self.loss_history.append(self.ls.detach().cpu().numpy())
        self.loss_history_u.append(u_loss.detach().cpu().numpy())
        self.loss_history_v.append(v_loss.detach().cpu().numpy())
        self.loss_history_f.append(f_loss.detach().cpu().numpy())
        self.loss_history_g.append(g_loss.detach().cpu().numpy())

        # track lam_u, lam_r history
        self.lam_u_log.append(self.lam_u.detach().cpu().numpy())
        self.lam_r_log.append(self.lam_r.detach().cpu().numpy())

        self.ls.backward()

        self.iter += 1
        if not self.iter % 1: 
            print('Iteration: {:}, Loss: {:0.6f}'.format(self.iter, self.ls))

        return self.ls
    
    def train(self):
        # # Adam training
        # while self.iter < 100:
        #     self.net.train()
        #     self.optimizer.step(self.closure)
        
        # # L-BFGS training
        # if self.iter == 100:
        #     print('--------Switching optimizer--------')
        #     print(self.optimizer)
        #     self.optimizer = torch.optim.LBFGS(self.net.parameters(), lr=1, max_iter=200000, max_eval=50000,history_size=50, tolerance_grad=1e-5, tolerance_change=0.5 * np.finfo(float).eps, line_search_fn="strong_wolfe")
        #     print(self.optimizer)
        #     print('-----------------------------------')
        self.net.train()
        self.optimizer.step(self.closure)
        self.scheduler.step()
         
    
    def compute_jacobian(self, f):
        J_list = []
        weights, biases = self.get_weights_and_biases(self.net)
        for i in len(weights):
            J_w = jacobian(f, weights[i])
            J_list.append(J_w)
        for i in len(biases):
            J_b = jacobian(f, biases[i])
            J_list.append(J_b)

        print("Computed Jacobian")
        return J_list

    def compute_ntk(self, J1, D1, J2, D2):
        # computes the NTK: K = J * J.T
        N = len(J1)

        ker = torch.zeros((D1, D2))
        for i in range(N):
            J1 = torch.reshape(J1[i], shape=(D1, -1))
            J2 = torch.reshape(J2[i], shape=(D2, -1))

            K = torch.matmul(J1, torch.transpose(J2, 0, 1))
            ker = ker + K

        print("Computed NTK")

        return ker

In [16]:
PINN = NS(x_train, y_train, t_train, u_train, v_train, kernel_size)
PINN.train()

RuntimeError: stack expects each tensor to be equal size, but got [512, 3] at entry 0 and [2, 512] at entry 1

In [10]:
kernel_size = 300
N_train = 5000
nu = 0.01 # Reynold's number

data = scipy.io.loadmat('C:\\Users\\Administrator\\Documents\\research\\fluid-sim\\data\\2d_navierstokes.mat')

usol = data['usol'] # sqrt(N) x sqrt(N) x T
vsol = data['vsol'] # sqrt(N) x sqrt(N) x T
psol = data['psol'] # sqrt(N) x sqrt(N) x T

# cut off first entries from usol, vsol, and psol because it is empty
usol = usol[:, :89, 1:] # cut off an extra row in y dimension
vsol = vsol[:89, :, 1:] # cut off an extra row in x dimension 
psol = psol[:89, :89, 1:]

x = np.arange(0, 1, 1/90)[:89]
y = np.arange(0, 1, 1/90)[:89]
t = np.arange(0, 4, 0.01).reshape(-1, 1)[:399]

X, Y, T = np.meshgrid(x, y, t, indexing='ij')
X = X.reshape(-1, 1)
Y = Y.reshape(-1, 1)
T = T.reshape(-1, 1)
X_star = np.concatenate((X, Y, T), axis=1).reshape(89, 89, 399, 3)

N = X_star.shape[0] * X_star.shape[1]
T = t.shape[0]

x_bc = np.concatenate((X_star[:, 0, :], X_star[:, -1, :], X_star[0, 1:-1, :], X_star[-1, 1:-1, :]), axis=0)
x_ic = X_star[:, :, 0].reshape(N, 3) # positions
x_bc = x_bc.reshape(x_bc.shape[0] * x_bc.shape[1], x_bc.shape[2])
x_icbc = np.vstack((x_ic, x_bc))

# extract initial and boundary conditions
u_ic = usol[:, :, 0]
u_bc = np.concatenate((usol[:, 0, :], usol[:, -1, :], usol[0, 1:-1, :], usol[-1, 1:-1, :]), axis=0)
u_icbc = np.hstack((u_ic.flatten(), u_bc.flatten())).reshape(-1, 1)

v_ic = vsol[:, :, 0]
v_bc = np.concatenate((vsol[:, 0, :], vsol[:, -1, :], vsol[0, 1:-1, :], vsol[-1, 1:-1, :]), axis=0)
v_icbc = np.hstack((v_ic.flatten(), v_bc.flatten())).reshape(-1, 1)

idx = np.random.choice(u_icbc.shape[0], N_train, replace=False)
x_train = x_icbc[idx][:, 0].reshape(-1, 1)
y_train = x_icbc[idx][:, 1].reshape(-1, 1)
t_train = x_icbc[idx][:, 2].reshape(-1, 1)

u_train = u_icbc[idx, :]
v_train = v_icbc[idx, :]
# p_train = p[idx, :]